In [1]:
# add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

We must make sure tha images are in a range we understand and have the instensity value we can exploit

In [6]:
image_dir = repo_path /'data/images/SET'
# get all images in dir
images = [image_dir / img for img in os.listdir(image_dir) if img.endswith('.tif')]
print(f'{len(images)} images were found')

# read images and get their shape, dtype, range


38 images were found
